## Separating Hyperplanes and Support Vector Machines

Separating Hyperplanes are a class of methods to construct linear decision boundaries that tries to separate the data into different classes. A hyperplane is defined by the equation

$$f(x) = \beta_0 + \beta^Tx = 0$$

In $\mathbb{R^2}$, this is simply a line. Based on the sign of $f(x_i)$ we can say which side of the hyperplane the point $x_i$ lies in. For binary classification problems with response variables taking $\{-1, 1\}$ as possible class values, we can construct a separating hyperplane such that if $f(x^*) > 0$, then $y_i = 1$ and when $f(x^*) < 0$, then $y_i = -1$ for any test observation $x^*$.


### Rosenblatt's Perceptron Learning Algorithm

This algorithm tries to find a separating hyperplane by minimizing the distance of misclassified points to the hyperplane decision boundary. The goal is then to minimize

$$D(\beta, \beta_0) = - \sum_{i \in \mathcal M} y_i (x_i^T \beta + \beta_0)$$

where $\mathcal M$ are the indices for the misclassified points, and the function on the right is proportional to the distance of the point from the decision boundary. The criterion here is piecewise and we can use stochastic gradient descent for minimizing it. We can visit the misclassified observations in some sequence and update $\beta$

\begin{align}
\beta & \leftarrow \beta + \rho (y_i x_i) \\
\beta_0 & \leftarrow \beta_0 + \rho (y_i)
\end{align}

where $\rho$ is the learning rate and $y_i x_i$ and $y_i$ appear as the gradient of $D$ with respect to $\beta, \beta_0$. This converges to a separating hyperplane if the classes are linearly separable. However, a potential problem is that there are multiple separating hyperplanes when the data is separable, and the converged solution will depend on the initial conditions. The solution to this is to add additional constraints on the separating hyperplane.


### Optimal Separating Hyperplanes

Optimal separating hyperplanes separates the two classes and also maximizes the distance of the closest points in either class to the hyperplane. The intuition behind this is that, when several separating hyperplanes are present, the one that's farthest from the points will generalize the best to test data. This is the solution to the optimization problem


$$\max_{\beta, \beta_0, \lVert \beta \rVert = 1} M$$
$$\text{subject to } y_i(x_i^T \beta + \beta_0) \ge M, i = 1, ..., n$$

The set of conditions ensure that each observation is at least a distance of $M$ from the decision boundary. This can be solved by implementing the constraint with the objective into a Lagrange primal function, and then solving the simpler optimization problem by converting to the dual problem. This is also known as the maximal margin hyperplane, since the algorithm maximizes the _margin_: which is the smallest perpendicular distance from the hyperplane to one of the observations. $M$ represents the margin of our hyperplane.


### Support Vector Classifiers

Support Vector Classifers are an extension of optimal separating hyperplanes to non-separable (overlapping) classes, by allowing a _soft margin_. When classes overlap in the feature space, we can construct a hyperplane that optimally separates the class by allowing some points to be on the wrong side of the margin (or even the hyperplane). We can modify the optimal separating hyperplanes optimization problem by introducing slack variables $\epsilon_1, ..., \epsilon_n$ to modify the constraint. A natural choice for the constraint would be

$$y_i(x_i^T \beta + \beta_)) \ge M - \epsilon_i$$

This measures the overlap in actual distance from the margin. However, this leads to a nonconvex optimzation problem. Instead, measuring overlap in relative distances $M(1 - \epsilon_i)$ leads to a convex optimization problem

$$\max_{\beta, \beta_0, \lVert \beta \rVert = 1, \epsilon_1, ..., \epsilon_n} M$$
$$\text{subject to } y_i(x_i^T \beta + \beta_0) \ge M(1 - \epsilon_i), i = 1, ..., n$$
$$\epsilon_i \ge 0$$
$$\sum_{i=1}^n \epsilon_i \le C$$

When $\epsilon_i = 0$, then the observation lies on the correct side of the margin. If $\epsilon_i > 0$, the observation lies on the wrong side of the margin and for $\epsilon_i > 1$, on the wrong side of the hyperplane. $C$ can be seen as the budget for the amount that the margin can be violated. For $C = 0$, this is simply the optimal separating hyperplane algorithm, where the model fits the data with low bias, but high variance. As $C$ increases, we become more tolerant of violation and the margin widens, thereby reducing the variance, but increasing bias.

The solution for $\beta$ has an interesting property, whereby it depends only on points that either lie on the margin or violate the margin. These points are known as ___support vectors___, as they can be seen to support the hyperplane. The hyperplane is robust to any observations that lie strictly on the correct side of the margin, and are affected only by a subset of the observations - the support vectors. The SVC computes on a linear decision boundary, and we can expand to nonlinear decision boundaries by expanding the feature space using basis functions.


### Support Vector Machines

We can select basis functions $h_m(x)$, $m = 1, ..., M$ and produce input features $h(x_i) = (h_1(x_i), ..., h_m(x_i))$ for $i = 1, ..., n$. We can then produce the nonlinear function by the regular support vector fitting procedure

$$f(x) = h(x)^T \beta + \beta_0$$

and classify observations based on the sign of $\hat f(x)$. The SV optimization problem can be reformulated such that the solution contains only inner products of the input features, without the observations themselves, and for particular choices of $h$, the inner products can be computed very cheaply. The solution for linear SVC can be written as

$$f(x) = \sum_{i=1}^n \alpha_i y_i \langle h(x_i), h(x_{i^\prime}) \rangle$$

The parameters $\beta_0, \alpha_i$ can be estimated by computing just the inner products between all pairs of training observations. In fact, we can just specify the ___kernel function___

$$K(x, x^\prime) = \langle h(x), h(x^\prime) \rangle$$

that specifies the inner products in the transformed space. Other choices for $K$ include

\begin{align}
\text{degree-} d \text{ polynomial} &: K(x, x^\prime) = (1 + \langle x, x^\prime \rangle)^d \\
\text{Radial basis} &: K(x, x^\prime) = \exp(- \gamma \lVert x - x^\prime \rVert^2) \\
\end{align}

The value of $C$ controls the smoothness of the fit: large values result in overfit wiggly boundaries, and small values result in smoother boundaries. Using kernels is more advantageous than simply enlarging the features space as kernels are computationall inexpensive, since we only need to calculate $K$ for $\binom n2$ pairs.


#### SVM as Loss + Penalty

The SVM optimization criterion can be rewritten as a penalization method

$$\min_{\beta_0, \beta} \sum_{i=1}^n [1 - y_i f(x_i)]_+ + \frac \lambda 2 \lVert \beta \rVert^2$$

where subscript $+$ indicates positive part. The penalty term is simply the ridge penalty term and plays a similar role. The loss function $L(y, f) = [1-yf]_+$ is called the ___hinge loss___ and is closely related to the logistic-regression loss function. This loss function is exactly zero for observations that are on the correct side of the margin. That is the rea 